In [ ]:
#У нас есть данные о связи lncRNA с участками генома - их можно проассоциировать с генами
#У фантома есть даные о дифф экспрессии генов после нокаута lncRNA - как бы эта РНК регулирует эти гены
#Или надо отобрать те гены, которые начали экспрессироваться после нокаута lncRNA?
# Что мы можем сделать?
#Мы должны взять из фантомных данных список генов, которые диффэкспрессируются у конкретной РНК и сравнить его
#с нашим списком генов для этой РНК - по идее они должны сильно пересекаться

In [ ]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#!pip install statsmodels
#!pip install bcbio-gff
#!pip install mygene
#!pip install h5py
#!pip install seaborn

In [ ]:
import pandas as pd
import pickle
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests

from BCBio import GFF
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
import mygene
import h5py

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
targets = [("H3K27ac", "_narrow"), ("H3K36me3", ""), 
           ("H3K4me1", "_narrow"), ("H3K4me2", "_narrow"), ("H3K4me3", "_narrow"), ("H3K79me2", ""), 
           ("H3K9ac", "_narrow"), ("H3K9me3", ""), ("H4K20me1", ""), ("H3K27me3", ""), ("methylation", "")]

In [ ]:
#Строим таблицу, где строки это все возможные lnc, для которых есть хотя бы одно значимое пересечение хотя бы для одной метки
#а столбцы мультииндекс по метке и по типу
def getTableForAllMarks(iMARGI=False, annotation_prefix=""):
    g_annotation_prefix = annotation_prefix + "_" if annotation_prefix else ""
    annotation_prefix = "_" + annotation_prefix if annotation_prefix else ""
    
    df_first = pd.read_csv("../data/all_marks/" + targets[0][0] + "/our_fantom" + annotation_prefix + "_genes_association_pvalues.tsv", sep="\t")
    df_first['lncRNAName'] = ["_".join([name] + i.split('_')[1:]) for i, name in zip(df_first['lncRNAId'], df_first['lncRNAName'])]
    if(iMARGI):
        g = pd.read_csv("../data/all_marks/" + targets[0][0] + "/" + g_annotation_prefix + "genometric_result_all_rnas.tsv", sep="\t")
        df_first = df_first[df_first['lncRNAId'].isin(g['lnc'])][['lncRNAName', 'pm_pvalue', 'mm_pvalue', 'pp_pvalue', 'mp_pvalue']]
    else:
        df_first = df_first[['lncRNAName', 'pm_pvalue', 'mm_pvalue', 'pp_pvalue', 'mp_pvalue']]
        
    for i in range(1, len(targets)):
        df_second = pd.read_csv("../data/all_marks/" + targets[i][0] + "/our_fantom" + annotation_prefix + "_genes_association_pvalues.tsv", sep="\t")
        df_second['lncRNAName'] = ["_".join([name] + i.split('_')[1:]) for i, name in zip(df_second['lncRNAId'], df_second['lncRNAName'])]
        if(iMARGI):
            g = pd.read_csv("../data/all_marks/" + targets[i][0] + "/" + g_annotation_prefix + "genometric_result_all_rnas.tsv", sep="\t")
            df_second = df_second[df_second['lncRNAId'].isin(g['lnc'])][['lncRNAName', 'pm_pvalue', 'mm_pvalue', 'pp_pvalue', 'mp_pvalue']]
        else:
            df_second = df_second[['lncRNAName', 'pm_pvalue', 'mm_pvalue', 'pp_pvalue', 'mp_pvalue']]
            
        df_first = df_first.merge(df_second, how="outer", left_on='lncRNAName', right_on='lncRNAName')
    
    df_first = df_first.set_index('lncRNAName')
    # del df_first.index.name
    df_first = df_first.applymap(lambda x: 0 if np.isnan(x) else -np.log10(x))
    columns_index_1 = ["H3K27ac", "H3K36me3", "H3K4me1", "H3K4me2", "H3K4me3", "H3K79me2", "H3K9ac", "H3K9me3", "H4K20me1", "H3K27me3", "Methylation"]
    columns_index_2 = ['wa', 'ea', 'wr', 'er']
    df_first.columns = pd.MultiIndex.from_product([columns_index_1, columns_index_2], names=['target', 'type'])
    
    np.unravel_index(np.argmax(df_first.values, axis=None), df_first.values.shape)
    
    #df_first = df_first.drop('EMX2OS')
    
    return df_first

In [ ]:
def getDataWithoutASOs(iMARGI=False, clusters=False):
    df = getTableForAllMarks(iMARGI=iMARGI, annotation_prefix="fantom_aso")
    df = df.mask(df < 1.3, 0)
    df['geneName'] = [i.split('_')[0] for i in df.index]
    b = df.groupby(['geneName']).apply(check_asos)
    b = b.drop(['geneName'], axis=1)
    b = b[b.astype(bool).sum(axis=1) > 0]
    if clusters:
        return b.apply(add_cluster, axis = 1, result_type="expand")
    else:
        return b

In [ ]:
def check_asos(df):
    count = pd.DataFrame({"nonzero_count" : df.astype(bool).sum(axis=0)})['nonzero_count']
    if df.shape[0] == 1:  # Нет нескольких ASO - 1
        return count
    else:
        s = pd.Series([2 if b else 0 for b in count > df.shape[0]/2], index=count.index)
        return s

In [ ]:
df1_1 = getDataWithoutASOs(iMARGI=True)
df1_2 = getDataWithoutASOs(iMARGI=True)
df1_1[df1_1 > 1] = 0
df1_2[df1_2 == 1] = 0
df1_2[df1_2 > 1] = 1

df2_1 = getDataWithoutASOs(iMARGI=False)
df2_2 = getDataWithoutASOs(iMARGI=False)
df2_1[df2_1 > 1] = 0
df2_2[df2_2 == 1] = 0
df2_2[df2_2 > 1] = 1

df1_1['sum'] = df1_1.sum(axis=1)
df1_2['sum'] = df1_2.sum(axis=1)

df2_1['sum'] = df2_1.sum(axis=1)
df2_2['sum'] = df2_2.sum(axis=1)

sum_1 = pd.DataFrame({"gene": df1_1.index, "ASO_1": df1_1['sum'], "ASO_>1": df1_2['sum']})
sum_2 = pd.DataFrame({"gene": df2_1.index, "ASO_1": df2_1['sum'], "ASO_>1": df2_2['sum']})

sum_1['IMARGI'] = "TRUE"
sum_1['INDEXES'] = sum_1['gene'].astype(str) +"_"+ sum_1["IMARGI"]
sum_1 = sum_1.reset_index(drop=True)
sum_1 = sum_1[['INDEXES', 'ASO_1', 'ASO_>1']]

sum_2['IMARGI'] = "FALSE"
sum_2['INDEXES'] = sum_2['gene'].astype(str) +"_"+ sum_2["IMARGI"]
sum_2 = sum_2.reset_index(drop=True)
sum_2 = sum_2[['INDEXES', 'ASO_1', 'ASO_>1']]

sum_0 = pd.concat([sum_1, sum_2])
sum_0 = sum_0.sort_values(by=['INDEXES'], ascending=False)


In [ ]:
fields = ['ASO_1', 'ASO_>1']
colors = ["#127d72", "#d93079"]
labels = ['ASO = 1', 'ASO > 1']# figure and axis
rcParams['figure.figsize'] = 25,28
rcParams["patch.force_edgecolor"] = True
fig, ax = plt.subplots(1, figsize=(20, 20))# plot bars
left = len(sum_0) * [0]
for idx, name in enumerate(fields):
    plt.barh(sum_0['INDEXES'], sum_0[name], left = left, color=colors[idx])
    left = left + sum_0[name]# title, legend, labels
plt.title('Title', loc='left', fontsize='xx-large')
plt.legend(labels, loc='upper right', bbox_to_anchor=(0.5, 0.53, 0.5, 0.5), ncol=2, fontsize='xx-large', frameon=False)
plt.xlabel('Number of associations', fontsize='x-large')
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)# adjust limits and draw grid lines
plt.ylim(-0.5, ax.get_yticks()[-1] + 0.5)
ax.set_axisbelow(True)
ax.xaxis.grid(color='gray', linestyle='dashed')
plt.show()

In [ ]:
fig = ax.get_figure()
#fig.patch.set_alpha(0)
fig.savefig("../images/barplot.png", bbox_inches='tight', pad_inches =0)